## Peakdeck: Choosing the Best Parameters
As part of the HIVDC collaboration with Alex Rives, we are interepreting ATACSeq data from the Littman Lab at NYU Med. In order to transform ATACSeq High Throughput Sequencing data into Transcription Factor Activity priors, we need to infer chromatin accessibility from peaks of sequencing reads. We do this by first aligning the Fastq data with Bowtie, then calling peaks with Kernel-Density-Estimator-based <a> Peakdeck </a href=http://www.ncbi.nlm.nih.gov/pubmed/24407222> 

### Pipeline overview
Fastq -> Sam -> Bam -> filtered Bam (removing Mitochondrial DNA) -> Sam -> Peakdeck output -> Bed -> Wig.

We also generate a tdf from the filtered bam file

We've pulled DNAseI hypersensitivity sites from Encode using the following protocol: Downloaded from http://hgdownload-test.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgDnaseMasterSites/releaseLatest/wgEncodeAwgDnaseMasterSites.bed.gz, then lifted over to hg38 using the chain file from http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg38.over.chain.gz

The bed master file was sorted and filtered to only have chromosomes whose names exist in the HG38 assembly and sorted

Running this on our cluster gives the following genome-wide output:

| depth | length | total | fraction  | 
|------|--------|-------|----|
| 0 | 2663458615 | 3099750718 | 0.859 |
| 1 | 436088249 | 3099750718 | 0.141 |
| 2 | 203511 | 3099750718 | 6.57e-05 |
| 3 | 343 | 3099750718 | 1.11e-07 |

This shows that the Encode hypersensitivy regions cover slightly under 15% of the genome, with a very small number of self-overlaps. 

### Sanity Check
Let's run this on the orignal (pre-lift over) hg19 bed file and verify that this depth signature is approximately the same. 

Genome-wide Output:

| depth | length | total | fraction  | 
|------|--------|-------|----|
| 0 | 2703081244 | 3137161264 | 0.862 |
| 1 | 434080020 | 3137161264 | 0.138 |

Therefore this ~200kb overlap of hypersensitivity sites is the result of conversion from hg19 to hg38

### Problem Setup
We will set the Encode hypersensitivy regions as the gold standard. Predicted peaks from peakdeck that overlap with these regions will be considered TPs, predicted peaks that don't overlap will be considered FPs, and hypersensitivy regions that don't overlap with any predicted peaks will be FNs. 

### TBD
We have data from 22 samples. Should their peaks be combined? Let's first do a quick sanity check to make sure there aren't outliers. We can look at the peakdeck output bed files for fraction of intersections out of all intervals, and do a heatmap of distance across all samples. 

In [ ]:

$BEDTOOLS merge -i accessible.sorted.bed > allpeaks.bed
